In [1]:
import pandas as pd
import datetime
import numpy as np
import matplotlib as mpl
import matplotlib.pyplot as plt
import seaborn as sns
import matplotlib.dates as mdates

import chart_studio.plotly as py
import cufflinks as cf
import plotly.express as px
import plotly.graph_objects as go
import plotly.figure_factory as ff

# Make Plotly work in your Jupyter Notebook
from plotly.offline import download_plotlyjs, init_notebook_mode, plot, iplot
init_notebook_mode(connected=True)
# Use Plotly locally
cf.go_offline()

%matplotlib inline

In [2]:
url = 'https://raw.githubusercontent.com/ngophuong/VNStock/main/excel_pow.csv'
df = pd.read_csv(url, index_col = 0)
df['datetime']=pd.to_datetime(df['<DTYYYYMMDD>'],format='%Y%m%d')
df['Week_no']= df['datetime'].dt.week
df['Year_no']= df['datetime'].dt.year
df['Period'] = df['Year_no'].astype(str) + str("-") + df['Week_no'].astype(str)
df.head(3)

<ipython-input-2-ae4d721cd850>:4: FutureWarning:

Series.dt.weekofyear and Series.dt.week have been deprecated.  Please use Series.dt.isocalendar().week instead.



,<DTYYYYMMDD>,<OpenFixed>,<HighFixed>,<LowFixed>,<CloseFixed>,<Volume>,<Open>,<High>,<Low>,<Close>,<VolumeDeal>,<VolumeFB>,<VolumeFS>,datetime,Week_no,Year_no,Period
<Ticker>,,,,,,,,,,,,,,,,,
POW,20210428,12.10,12.55,12.10,12.2,7027100,12.10,12.55,12.10,12.2,0,621800,378000,2021-04-28,17,2021,2021-17
POW,20210427,12.10,12.35,11.90,12.1,8043200,12.10,12.35,11.90,12.1,40000,641600,198600,2021-04-27,17,2021,2021-17
POW,20210426,12.75,12.75,12.05,12.1,11207300,12.75,12.75,12.05,12.1,50000,447600,848400,2021-04-26,17,2021,2021-17


In [3]:
df =df[['<Close>','datetime','<Volume>','Period','Year_no']]
df = df.head(1500)

In [4]:
#plot linechart for stock price
short_rolling = df['<Close>'].rolling(window=20).mean()

ema_long = df['<Close>'].ewm(span=50, adjust=False).mean()
ema_short = df['<Close>'].ewm(span=20, adjust=False).mean()
line_price = df['<Close>']
fig = go.Figure()
#px.line(df_hpg, x='datetime', y='<Close>', labels={'x':'Date', 'y':'Price'})
fig.update_layout(  autosize=False,   width=1200,    height=800)
fig.add_trace(go.Scatter(x=df.datetime, y=line_price, 
                        mode='lines', name='POW'))

fig.add_trace(go.Scatter(x=df.datetime, y=ema_short, 
                        mode='lines', name='Span 20-days EMA'))

fig.add_trace(go.Scatter(x=df.datetime, y=ema_long, 
                        mode='lines', name='Span 50-days EMA'))


In [19]:
#px.violin(df_tips, y="tip", x="smoker", color="sex", box=True, points="all",
#          hover_data=df_tips.columns)

fig = go.Figure()

px.violin(df,x='Year_no',y='<Close>',box =True, points='all',
         hover_data=df.columns, title = 'STOCK Violin chart',width=1200,    height=800 )

In [6]:
df.tail(3)

,<Close>,datetime,<Volume>,Period,Year_no
<Ticker>,,,,,
POW,17.2,2018-03-08,6824944,2018-10,2018
POW,16.8,2018-03-07,7174392,2018-10,2018
POW,17.8,2018-03-06,14922100,2018-10,2018


In [7]:
#HDFC_df[‘Day_Perc_Change’] = HDFC_df[‘Adj Close’].pct_change()*100
df['Daily_return']=df['<Close>'].pct_change()*100
df.dropna(axis = 0, inplace = True)
df.head(3)

,<Close>,datetime,<Volume>,Period,Year_no,Daily_return
<Ticker>,,,,,,
POW,12.1,2021-04-27,8043200,2021-17,2021,-0.819672
POW,12.1,2021-04-26,11207300,2021-17,2021,0.000000
POW,12.6,2021-04-23,14126600,2021-16,2021,4.132231


In [20]:
fig = go.Figure()
px.line(df, x='datetime', y='Daily_return', labels={'x':'Date', 'y':'Daily_Return_%'},width=1200,    height=800)

In [9]:
df['Daily_return'].describe()

count    779.000000
mean       0.073072
std        2.216537
min       -9.230769
25%       -0.978795
50%        0.000000
75%        1.203887
max        7.638889
Name: Daily_return, dtype: float64

In [21]:
fig = go.Figure()
px.histogram(df, x="Daily_return",histnorm='percent', width=1200,    height=800)
#ff.create_distplot(df, group_labels)

In [11]:
df['<Close>'].describe()

count    779.000000
mean      12.862362
std        2.283988
min        7.090000
25%       10.525000
50%       13.150000
75%       14.600000
max       17.800000
Name: <Close>, dtype: float64

In [22]:
dfig = go.Figure()
px.histogram(df.head(60), x="Daily_return",histnorm='percent', width=1200,    height=800)

In [13]:


df['Volumn'] = df['<Volume>']/1000000
Volumnshort_rolling = df['Volumn'].rolling(window=7).mean()

fig = go.Figure()
fig.update_layout(  autosize=False,   width=1200,    height=800)
fig.add_trace(go.Scatter(x=df.datetime, y=df['Daily_return'], 
                        mode='lines', name='Dailly_Retun'))
fig.add_trace(go.Scatter(x=df.datetime, y=df['Volumn'], 
                        mode='lines+markers', name='Volumn'))

fig.add_trace(go.Scatter(x=df.datetime, y=Volumnshort_rolling, 
                      mode ='lines', name='Rolling_volumn'))

fig.add_trace(go.Scatter(x=df.datetime, y=df['<Close>'], 
                        mode='lines+markers', name='Price'))

In [14]:
df['Volumn'].head(15).describe()

count    15.000000
mean     16.775933
std       7.655246
min       8.043200
25%      12.199150
50%      14.126600
75%      19.300700
max      39.387800
Name: Volumn, dtype: float64

In [15]:
df['Volumn'].head(10).describe()

count    10.000000
mean     14.804630
std       4.539208
min       8.043200
25%      11.312750
50%      14.490500
75%      16.981375
max      21.614100
Name: Volumn, dtype: float64

In [16]:
df.head(3)

,<Close>,datetime,<Volume>,Period,Year_no,Daily_return,Volumn
<Ticker>,,,,,,,
POW,12.1,2021-04-27,8043200,2021-17,2021,-0.819672,8.0432
POW,12.1,2021-04-26,11207300,2021-17,2021,0.000000,11.2073
POW,12.6,2021-04-23,14126600,2021-16,2021,4.132231,14.1266
